<a href="https://colab.research.google.com/github/cruz-marco/pyspark_course/blob/main/pyspark_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = '/content/spark-3.2.3-bin-hadoop3.2'

!pip install -q findspark

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# Spark SQL

- O Spark permite també utilizar tabelas dentro de banco de dados um ambiente que se assemelha ao de um banco de dados relacional comum (Postgres, MySQL)

- As tabelas são exatamente iguais em estrutura e funcionalidades aos bancos de dado SQL. Diferindo dos DataFrames, as tabelas são objetos que persistem depois que a sessão do Spark é finalizada.

- Tabelas e DataFrames são totalmente interoperáveis no ambiente Spark, logo, podemos fazer o cast de um tipo de estrutura para outro sem muito problema.

- Apesar de sermos completamente livres para fazer consultas e gerenciamento de bancos de dados e tabelas pelo spark, o retorno no framework do PySpark é sempre em um objeto DataFrame, o que não significa que a tabela seja mutável, mas que a saída daquela consulta usa o tipo de dado DataFrame.

## 1. Tipos de Tabelas:
> **Gerenciadas (Managed)**: Spark gerendia os dados e os metadados deta tabela. Se apagarmos esta tabela, ela desaparece por completo (dados e metadados desaparecem). Ficam armazenadas no Warehouse do Spark.

> **Não Gerenciadas (External)**: É uma tabelo onde apenas os metadados são gerenciados pelo Spark e os dados em si estão localizados um um ludar diferente do Spark Warehouse.

## 2. Views:
As views são apelidos que podemos dar a uma tabela que é criada a partir de uma consulta. Usando joins ou criando novas colunas com os dados, por exemplo. O uso de views facilita o trabalho quando precisamos executar uma consulta mais complexa e que necessita de um aninhamento de selects, por exemplo.
### Podem ser de dois tipos:
> **Globais**: Visíveis em todas as sessões.

> **Sessão**: Visíveis apenas na sessão atual, sendo destruídas quando a mesma é encerrada.




### Criando o dataframe despachantes para ser usado nos exemplos a seguir.

In [13]:
from pyspark.sql.types import *

arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"

despachantes = spark.read.csv("/content/drive/MyDrive/Datasets/pyspark_course/despachantes.csv", 
                              header=False, schema=arqschema)

### Mostrando e criando os bancos de dados.

In [5]:
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [6]:
spark.sql('create database desp').show() # Criando o banco de dados DESP

++
||
++
++



In [7]:
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
|     desp|
+---------+



In [8]:
spark.sql('use desp') # Selecionando o banco de dados DESP para ser usado.

DataFrame[]

- Com o banco de dados criado e selecionado, podemos escrever o DataFrame *despachantes* completo como uma tabela dentro do banco de dados DESP com apenas umalinha de código.

In [9]:
despachantes.write.saveAsTable("Despachantes") 

### Mostrando a recém criada **TABELA** Despachantes usando um select clássico SQL.

In [10]:
spark.sql('select * from Despachantes').show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



### Mostrando as tabelas contidas no banco de dados Desp

In [11]:
spark.sql('show tables').show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|     desp|despachantes|      false|
+---------+------------+-----------+



### Inserindo o método "*.mode()*" podemos modificar o modo de escrita da tabela. Por exemplo, usando OVERWRITE para sobrescrever ou APPEND para adicionar mais linhas mantendo as que já estão persistidas na tabela..

In [12]:
despachantes.write.mode('overwrite').saveAsTable("Despachantes")